In [1]:
import tensorflow as tf
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import mplfinance as mpf
import talib as talib
import numpy as np
import data as ds
import common as common
import os as os
import math as math
import datetime as datetime
import scipy as sp
import itertools  as itertools
import multiprocessing as mp
from os import listdir, walk
from itertools import repeat
from mplfinance.original_flavor import candlestick_ohlc
from datetime import datetime, timedelta
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, pairwise, mean_squared_error, mean_absolute_error
from scipy import stats
from pprint import pprint

In [2]:
# 0.1 环境设定
pd.options.mode.chained_assignment = None

# 0.2 不让程序占满 GPU 内存
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
# 1.0 重構數據集
def reshape_dataframe(df0):
    df0.fillna(0, inplace=True)
    df0.replace([np.inf, -np.inf], 0, inplace=True)
    df0.columns = ['udate', 'High', 'Low', 'Open', 'Close', 'Volume']
    types2 = {'udate': 'object', 'High': 'float64', 'Low': 'float64', 'Open': 'float64', 'Close': 'float64', 'Volume': 'int64'}
    df0.astype(types2).dtypes
    df0_1 = df0.copy(deep=True)
    error_row = []
    for k, v in df0.iterrows():
        if not pd.isnull(df0['udate'].iloc[k]) and df0['udate'].iloc[k] > 0:
            stime = str(int(df0['udate'].iloc[k]))
            df0_1['udate'].iloc[k] = datetime(year=2020, month=int(stime[-8:-6]), day=int(stime[-6:-4]), hour=int(stime[-4:-2]), minute=int(stime[-2:]), second=0)
        else:
            error_row.append(k)
    df0_1.drop(df0_1.index[error_row], inplace=True)
    df0_1.udate = pd.to_datetime(df0_1.udate)
    df0_1.index = pd.to_datetime(df0_1.udate)
    
    # 1.0.1 數據有效性檢查
    for k, v in types2.items():
        if (df0_1[k].isin([np.nan]).any().any()):
            print(k+' obtains nan')
        if (df0_1[k].isin([0]).any().any()):
            print(k+' obtains 0')
    is_contain_null = df0_1.isnull().sum()

    return df0_1

# 1.1 数据源
files = []
"""
file_1 = os.path.abspath(os.path.join('data', 'nq', 'data', 'nq-20201105.csv'))
df1_1 = reshape_dataframe(pd.read_csv(file_1))
files.append(df1_1)

file_2 = os.path.abspath(os.path.join('data', 'nq', 'data', 'nq-20201105_Aug_Sep.csv'))
df1_2 = reshape_dataframe(pd.read_csv(file_2))
files.append(df1_2)

file_3 = os.path.abspath(os.path.join('data', 'nq', 'data', 'nq-20201110.csv'))
df1_3 = reshape_dataframe(pd.read_csv(file_3))
files.append(df1_3)

file_4 = os.path.abspath(os.path.join('data', 'nq', 'data', 'nq-20201111.csv'))
df1_4 = reshape_dataframe(pd.read_csv(file_4))
files.append(df1_4)
"""

for i in range(2, 10):
    month1 = str(i).zfill(2)
    file_5 = os.path.abspath(os.path.join('data', 'nq', 'data', 'nq-20-'+month1+'.csv'))
    df1_5 = reshape_dataframe(pd.read_csv(file_5))
    files.append(df1_5)

df2 = pd.concat(files, ignore_index=False)

# 1.2 刪除重覆index
df2 = df2.groupby(df2.index).first()

# 1.3 排序
df2.sort_index(axis=0, ascending=True, inplace=True)
df2

Volume obtains 0
Volume obtains 0
Volume obtains 0
Volume obtains 0
Volume obtains 0
Volume obtains 0
Volume obtains 0
Volume obtains 0


,udate,High,Low,Open,Close,Volume
udate,,,,,,
2020-02-16 17:01:00,2020-02-16 17:01:00,9650.00,9657.00,9649.00,9654.00,459
2020-02-16 17:02:00,2020-02-16 17:02:00,9654.00,9656.75,9653.00,9653.75,468
2020-02-16 17:03:00,2020-02-16 17:03:00,9653.75,9654.00,9653.00,9653.50,168
2020-02-16 17:04:00,2020-02-16 17:04:00,9653.50,9657.50,9653.25,9656.50,356
2020-02-16 17:05:00,2020-02-16 17:05:00,9656.50,9658.25,9655.50,9657.25,260
...,...,...,...,...,...,...
2020-09-15 16:55:00,2020-09-15 16:55:00,11462.50,11462.50,11462.50,11462.50,0
2020-09-15 16:56:00,2020-09-15 16:56:00,11462.50,11462.50,11462.50,11462.50,0
2020-09-15 16:57:00,2020-09-15 16:57:00,11462.50,11462.50,11462.50,11462.50,0


In [4]:
# 2.0 分包 (日子)
def separate_daily(df2_1, return_type):
    data1 = {}
    # 2.1交易日
    days1 = list(dict.fromkeys([v.date() for v in df2_1['udate']]))
    for day in days1:
        # 2.2 交易時間
        day_start = datetime(day.year, day.month, day.day, 17, 0, 0)
        day2 = day + timedelta(days=1)
        day_end = datetime(day2.year, day2.month, day2.day, 16, 0, 0)
        mask = ((df2_1['udate'] >= day_start) & (df2_1['udate'] <= day_end))
        df2_2 = df2_1.loc[mask]
        if (df2_2.shape[0] > 1):
            data1[day] = df2_2
    # 2.3 合併
    df2_3 = pd.DataFrame()
    for k, df2_4 in data1.items():
        df2_3 = pd.concat([df2_3, df2_4], axis=0, join='outer', ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, copy=True)
    # 2.4 返回類型
    if return_type=='df':
        return df2_3
    elif return_type=='dict':
        return data1

df2_5 = df2.copy(deep=True)
df3 = separate_daily(df2_5, 'df')
df3

,udate,High,Low,Open,Close,Volume
udate,,,,,,
2020-02-16 17:01:00,2020-02-16 17:01:00,9650.00,9657.00,9649.00,9654.00,459
2020-02-16 17:02:00,2020-02-16 17:02:00,9654.00,9656.75,9653.00,9653.75,468
2020-02-16 17:03:00,2020-02-16 17:03:00,9653.75,9654.00,9653.00,9653.50,168
2020-02-16 17:04:00,2020-02-16 17:04:00,9653.50,9657.50,9653.25,9656.50,356
2020-02-16 17:05:00,2020-02-16 17:05:00,9656.50,9658.25,9655.50,9657.25,260
...,...,...,...,...,...,...
2020-09-15 15:56:00,2020-09-15 15:56:00,11462.25,11462.25,11462.25,11462.25,0
2020-09-15 15:57:00,2020-09-15 15:57:00,11462.25,11465.00,11462.25,11464.00,24
2020-09-15 15:58:00,2020-09-15 15:58:00,11464.00,11465.00,11463.75,11464.00,59


In [5]:
# 3.0 技術指標
highs = np.array(df3['High'], dtype='float')
lows = np.array(df3['Low'], dtype='float')
opens = np.array(df3['Open'], dtype='float')
closes = np.array(df3['Close'], dtype='float')
vols = np.array(df3['Volume'], dtype='float')
# 3.1 SMA 均線
for v in [5, 10, 20, 50, 100]:
    df3['sma-'+str(v)]= talib.SMA(closes, timeperiod=v)
# 3.2 Bollinger 保力加
df3['upper-band'], df3['middle-band'], df3['lower-band'] = talib.BBANDS(closes, timeperiod=20*1.5, nbdevup=2, nbdevdn=2, matype=0)
# 3.3 %B %保力加
df3['%b'] = (df3['Close']-df3['lower-band'])/(df3['upper-band']-df3['lower-band'])*100
df3['%b-high']  = common.percentB_belowzero(df3['%b'], df3['Close']) 
df3['%b-low'] = common.percentB_aboveone(df3['%b'], df3['Close'])
# 3.4 VOL EMA
df3['vol-ema5'] = talib.EMA(vols, timeperiod=5*4)
# 3.5 P-SAR 抛物线
df3['p-sar'] = talib.SAR(highs, lows, acceleration=0.02, maximum=0.2)
# 3.6 VWAP 成交量加權平均價格
period = [5, 10, 20, 50, 100]
for v in period:
    df3['typical-price'] = (df3['High'] + df3['Low'] + df3['Close']) / 3
    df3['turnover'] = df3['typical-price'] * df3['Volume']
    df3['cum-turnover-'+str(v)] = df3['turnover'].rolling(window=v).sum()
    df3['cum-volume-'+str(v)] = df3['Volume'].rolling(window=v).sum()
    df3['vwap-'+str(v)] = df3['cum-turnover-'+str(v)] / df3['cum-volume-'+str(v)]
    df3['vwap-'+str(v)] = df3['vwap-'+str(v)].replace([np.inf, -np.inf], 0)
    df3['vwap-'+str(v)].fillna(0, inplace=True)
    drop_list_1 = ['turnover', 'typical-price', 'cum-turnover-'+str(v), 'cum-volume-'+str(v)]
    df3.drop(drop_list_1, axis=1, inplace=True)
# 2.7 MACD
df3['macd'], df3['macdsignal'], df3['macdhist'] = talib.MACD(closes, fastperiod=12, slowperiod=26, signalperiod=9*40)
# 2.8 KDJ
df3['k-kdj'], df3['d-kdj'], df3['j-kdj'] = common.kdj(highs, lows, closes, window_size=20)
df3['diff-kdj'] = df3['k-kdj']-df3['d-kdj']
df3['j-kdj'].loc[((df3['j-kdj'] > 20) & (df3['j-kdj'] < 100))] = 0
df3

,udate,High,Low,Open,Close,Volume,sma-5,sma-10,sma-20,sma-50,...,vwap-20,vwap-50,vwap-100,macd,macdsignal,macdhist,k-kdj,d-kdj,j-kdj,diff-kdj
udate,,,,,,,,,,,,,,,,,,,,,
2020-02-16 17:01:00,2020-02-16 17:01:00,9650.00,9657.00,9649.00,9654.00,459,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-02-16 17:02:00,2020-02-16 17:02:00,9654.00,9656.75,9653.00,9653.75,468,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-02-16 17:03:00,2020-02-16 17:03:00,9653.75,9654.00,9653.00,9653.50,168,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-02-16 17:04:00,2020-02-16 17:04:00,9653.50,9657.50,9653.25,9656.50,356,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-02-16 17:05:00,2020-02-16 17:05:00,9656.50,9658.25,9655.50,9657.25,260,9655.00,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-15 15:56:00,2020-09-15 15:56:00,11462.25,11462.25,11462.25,11462.25,0,11462.40,11461.975,11461.9875,11458.230,...,11462.737436,11444.671751,11415.538590,1.554868,0.785650,0.769218,76.625582,35.926251,158.024242,40.699330
2020-09-15 15:57:00,2020-09-15 15:57:00,11462.25,11465.00,11462.25,11464.00,24,11462.75,11462.400,11461.9250,11459.225,...,11462.701493,11446.494981,11415.282245,1.590913,0.790111,0.800802,77.915387,36.985476,159.775209,40.929911
2020-09-15 15:58:00,2020-09-15 15:58:00,11464.00,11465.00,11463.75,11464.00,59,11463.30,11462.625,11461.7875,11460.035,...,11462.834206,11448.283255,11415.186842,1.601024,0.794604,0.806420,79.181038,38.072977,161.397159,41.108061


In [6]:
# 4.0 draw chart
drop_list_2 = []
is_render_chart = False

def draw_worker(df4_2, day):
    try:
        # 4.1 style
        style = mpf.make_mpf_style(base_mpf_style='charles', rc={'font.size':6})
        # 4.2 addplot
        apds = [mpf.make_addplot(df4_2['lower-band'],panel=0,color='orange',linestyle='solid'),
                mpf.make_addplot(df4_2['upper-band'],panel=0,color='bisque',linestyle='solid'),
                mpf.make_addplot(df4_2['vwap-50'].replace(0, np.nan),panel=0,color='aqua',linestyle='solid'),
                mpf.make_addplot(df4_2['%b-low'],type='scatter',markersize=20,marker='v',panel=0),
                mpf.make_addplot(df4_2['%b-high'],type='scatter',markersize=20,marker='^',panel=0),
                mpf.make_addplot(df4_2['p-sar'],scatter=True,markersize=1,marker='*',panel=0,color='blueviolet'),
                #
                mpf.make_addplot(df4_2['vol-ema5'],panel=1,color='orange'),
                #
                mpf.make_addplot(df4_2['macd'],panel=2,color='orange'),
                mpf.make_addplot(df4_2['macdsignal'],panel=2,color='violet'),
                mpf.make_addplot(df4_2['macdhist'],panel=2,type='bar',color='dimgray'),
                #
                mpf.make_addplot(df4_2['k-kdj'],panel=3,color='orange'),
                mpf.make_addplot(df4_2['d-kdj'],panel=3,color='violet'),
                mpf.make_addplot(df4_2['j-kdj'],panel=3,color='aqua'),
                mpf.make_addplot(df4_2['diff-kdj'],panel=3,type='bar',color='dimgray')]
        # 4.3 draw
        mpf.plot(df4_2, type='candle', addplot=apds, style=style, ylabel='', ylabel_lower='', volume=True, figscale=0.5, xrotation=0, datetime_format="%H:%M", show_nontrading=False, tight_layout=True, savefig='./data/img-nq/features/'+day.strftime('%m-%d-%Y'))
        print('finish draw:', df4_2.shape, df4_2['udate'].iloc[0], df4_2['udate'].iloc[-1])
    except:
        print('do not draw:', k, '\n')

# 4.5 multi processing
pool = mp.Pool(processes=4, maxtasksperchild=4)
# 4.6 draw
df4 = df3.copy(deep=True)
data2 = separate_daily(df4, 'dict')
for k, df4_1 in data2.items():
    # 4.7 drop error day
    if df4_1['Volume'].shape[0] == df4_1['Volume'].isin([0]).sum() or df4_1['Volume'].shape[0] <= 100:
        drop_list_2.append(k)
    # 4.8 draw chart
    elif (is_render_chart):
        pool.apply_async(draw_worker, args=(df4_1, k))
# 4.9 kill multi processing
pool.close()
pool.join()

for k in drop_list_2:
    data2.pop(k, None)
    print('excpet & delete: ', k)

excpet & delete:  2020-02-16
excpet & delete:  2020-02-18
excpet & delete:  2020-02-19
excpet & delete:  2020-02-20


In [7]:
# 5.0 合併
df5 = pd.DataFrame()
for k, df5_1 in data2.items():
    df5 = pd.concat([df5, df5_1], axis=0, join='outer', ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, copy=True)

# 5.1 清洗
df5_3 = df5.copy(deep=True)
drop_list_3 = ['High', 'Low', 'Open', 'Volume', 'macd', 'macdsignal', 'middle-band', 'k-kdj', 'd-kdj', 'vol-ema5', 'sma-10', 'vwap-10']
df5_3.drop(drop_list_3, axis=1, inplace=True)
df5_3.fillna(0, inplace=True)
df5_3 = df5_3.round(2)

# 5.2 檢查
is_contain_null = df5_3.isnull().sum()
is_contain_nan = df5_3.isna().sum()
is_contain_inf = df5_3.isin([np.nan]).sum()
print('数据集: Total dataset has {} samples, and {} features.'.format(df5_3.shape[0], df5_3.shape[1])) # df5_3.info()

# 5.3 儲存
path_data = os.path.abspath(os.path.join('data', 'nq', 'clean-data', 'nq-clean-data-with-features.csv'))
if os.path.exists(path_data):
    os.remove(path_data)
df5_3.to_csv(path_data)

# 5.4.1 分包 (日子)
data3 = separate_daily(df5_3, 'dict')
# 5.3.2 正則化
normalization_days = 10
days2 = np.array(list(data3.keys()))
len_days2 = (days2.shape[0]//normalization_days)+1
data3_1 = []
scalers = {}
for i in range(len_days2):
    try:
        start1, end1 = i*normalization_days, (i+1)*normalization_days
        if end1 >= days2.shape[0]:
            end1 = days2.shape[0]-1
        # 5.3.3 每X日集合
        day_start, day_end = days2[start1], days2[end1]
        day_start2 = datetime(day_start.year, day_start.month, day_start.day, 17, 0, 0)
        day_end2 = datetime(day_end.year, day_end.month, day_end.day, 16, 0, 0)
        mask = ((df5_3['udate'] >= day_start2) & (df5_3['udate'] <= day_end2))
        df5_4 = df5_3[mask]
        # 5.4.4 正則代集合
        min_max_scaler = MinMaxScaler(feature_range=(-0.99, 0.99))
        df5_5 = df5_4.drop(['udate'], axis=1)
        min_max_scaler.fit(df5_5)
        scalers[i] = {'scaler': min_max_scaler, 'day_start': day_start2, 'day_end': day_end2}
        df5_6 = min_max_scaler.transform(df5_5)
        df5_6 = pd.DataFrame(df5_6, columns=df5_5.columns, index=df5_5.index)
        df5_6['udate'] = df5_4['udate']
        # 5.4.5 合併
        data3_1.append(df5_6)
    except:
        print('except: ', start1, end1, len(days2))

# 5.4.6 合併
df5_7 = pd.DataFrame()
for df5_8 in data3_1:
    df5_7 = pd.concat([df5_7, df5_8], axis=0, join='outer', ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, copy=True)
df5_7

数据集: Total dataset has 199175 samples, and 19 features.


,Close,sma-5,sma-20,sma-50,sma-100,upper-band,lower-band,%b,%b-high,%b-low,p-sar,vwap-5,vwap-20,vwap-50,vwap-100,macdhist,j-kdj,diff-kdj,udate
udate,,,,,,,,,,,,,,,,,,,
2020-02-23 17:00:00,0.970192,0.990000,0.990000,0.990000,0.990000,0.906962,0.990000,-0.977270,0.99,-0.99,0.990000,0.990000,0.990000,0.990000,0.990000,0.114264,-0.267126,0.044846,2020-02-23 17:00:00
2020-02-23 17:01:00,0.979085,0.921577,0.971608,0.979392,0.985862,0.939174,0.930054,-0.641074,-0.99,-0.99,0.979085,0.952584,0.956905,0.959034,0.967751,0.114264,-0.267126,0.044846,2020-02-23 17:01:00
2020-02-23 17:02:00,0.983532,0.853936,0.953509,0.968866,0.981765,0.959125,0.883882,-0.488656,-0.99,-0.99,0.914989,0.936584,0.936244,0.940199,0.952357,0.114264,-0.267126,0.044846,2020-02-23 17:02:00
2020-02-23 17:03:00,0.990000,0.788074,0.935828,0.958407,0.977667,0.971551,0.845919,-0.391878,-0.99,-0.99,0.863439,0.921215,0.923321,0.928370,0.942080,0.114264,-0.267126,0.044846,2020-02-23 17:03:00
2020-02-23 17:04:00,0.969788,0.717802,0.917310,0.947623,0.973410,0.981126,0.809544,-0.353352,-0.99,-0.99,0.822202,0.900003,0.907189,0.910841,0.926422,0.114264,-0.267126,0.044846,2020-02-23 17:04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-14 15:56:00,0.049834,0.043128,0.040296,0.045414,0.004280,-0.046934,0.107320,-0.299660,-0.99,-0.99,0.052463,0.939630,0.940731,0.943214,0.943222,-0.048350,-0.418999,-0.105118,2020-09-14 15:56:00
2020-09-14 15:57:00,0.046694,0.041999,0.039259,0.043795,0.004425,-0.055252,0.109867,-0.340968,-0.99,-0.99,0.048293,0.939606,0.940694,0.943104,0.943243,-0.046121,-0.418999,-0.140626,2020-09-14 15:57:00
2020-09-14 15:58:00,0.054542,0.042805,0.038121,0.042796,0.005078,-0.061475,0.111911,-0.242019,-0.99,-0.99,0.044861,0.939623,0.940637,0.943022,0.943273,-0.037482,-0.418999,-0.167704,2020-09-14 15:58:00


In [8]:
# 5.5.1 訓練集, 測試集, 數據集
data4 = separate_daily(df5_7, 'dict')
days3 = np.array(list(data4.keys()))
train_set, test_set, valid_set = days3[0:-55], days3[-55:-15], days3[-15:-1]
x_train, y_train, date_train = np.array([]), np.array([]), []
x_test, y_test, date_test = np.array([]), np.array([]), []
x_valid, y_valid, date_valid = np.array([]), np.array([]), []
# 5.5.2 窗口步長
t_pus_no = 5
window_size = 100

for day, df6 in data4.items():
    df6.drop(['udate'], axis=1, inplace=True)
    # 5.5.3 窗口
    no_max = df6.shape[0]-t_pus_no
    x_data, y_data = [], []
    for i in range(window_size, no_max):
        start, end = i-window_size, i
        # y label
        temp_0 = df6.iloc[end: end+t_pus_no]
        y_data.append(temp_0['Close'])
        # x matrix
        temp_1 = df6.iloc[start: end]
        x_data.append(temp_1)
        # date
        days4 = temp_0.index.tolist()
        if day in train_set:
            date_train.append(days4)
        elif day in test_set:
            date_test.append(days4)
        elif day in valid_set:
            date_valid.append(days4)
    x_data, y_data = np.array(x_data), np.array(y_data)
    # print(day, x_data.shape, y_data.shape)
    # 5.5.4 分集1
    if day in train_set and x_train.any() and y_train.any():
        x_train = np.concatenate((x_train, x_data), axis=0)
        y_train = np.concatenate((y_train, y_data), axis=0)
    elif day in test_set and x_test.any() and y_test.any():
        x_test = np.concatenate((x_test, x_data), axis=0)
        y_test = np.concatenate((y_test, y_data), axis=0)
    elif day in valid_set and x_valid.any() and y_valid.any():
        x_valid = np.concatenate((x_valid, x_data), axis=0)
        y_valid = np.concatenate((y_valid, y_data), axis=0)
    # 5.5.5 分集2
    if day in train_set and not x_train.any() and not y_train.any():
        x_train, y_train = x_data, y_data
    elif day in test_set and not x_test.any() and not y_test.any():
        x_test, y_test = x_data, y_data
    elif day in valid_set and not x_valid.any() and not y_valid.any():
        x_valid, y_valid = x_data, y_data

# no_batches, timesteps, no_features
print('訓練集: X_train Data: {}, Y_train Data: {}, Date_Train: {}'.format(x_train.shape, y_train.shape, len(date_train)))
print('測試集: X_Test Data: {}, Y_Test Data: {}, Date_Test: {}'.format(x_test.shape, y_test.shape, len(date_test)))
print('验证集: X_Valid Data: {}, Y_Valid Data: {}, Date_Valid: {}'.format(x_valid.shape, y_valid.shape, len(date_valid)))

訓練集: X_train Data: (113467, 100, 18), Y_train Data: (113467, 5), Date_Train: 113467
測試集: X_Test Data: (50206, 100, 18), Y_Test Data: (50206, 5), Date_Test: 50206
验证集: X_Valid Data: (17650, 100, 18), Y_Valid Data: (17650, 5), Date_Valid: 17650


In [ ]:
# 6.1.1 模型参数
batch_size = 1024
epochs = 2
units = 128
verbose = 1
no_batches = x_train.shape[0]
timesteps = x_train.shape[1]
no_features = x_train.shape[2]
batch_input_shape = (timesteps, no_features)

# 6.1.2 日志参数
prefix = 'nq-lstm'
cur_time = datetime.now().strftime("%Y%m%d-%H%M%S")

# 6.2 模型 
# activation=softsign/tanh
model = tf.keras.Sequential()
model.add(LSTM(units=units, recurrent_activation='sigmoid', activation='tanh', unroll=False, use_bias=True, 
                               recurrent_dropout=0, return_sequences=True, input_shape=batch_input_shape))
model.add(Dropout(rate=0.2))

model.add(LSTM(units=units, activation='tanh',return_sequences=True, input_shape=batch_input_shape))
model.add(Dropout(rate=0.1))

model.add(LSTM(units=units, activation='tanh',return_sequences=True, input_shape=batch_input_shape))
model.add(Dropout(rate=0.2))

model.add(LSTM(units=units, activation='tanh',return_sequences=True, input_shape=batch_input_shape))
model.add(Dropout(rate=0.1))

model.add(LSTM(units=units, activation='tanh',return_sequences=False, input_shape=batch_input_shape))
model.add(Dropout(rate=0.2))

model.add(Dense(units=t_pus_no))
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.summary()

# 6.3.1 check point
checkpoint_dir = './training_checkpoints/'+ prefix +'-' + cur_time
os.mkdir(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True)

# 6.3.2 tensor board
log_dir = os.path.join('./logs/fit/'+ prefix +'-') + cur_time
os.mkdir(log_dir)
tensor_board_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# 6.4 fit model
history_model = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test), shuffle=True,
                          verbose=verbose, callbacks=[checkpoint_callback, tensor_board_callback])

# 6.5 save model
model_path = "./saved_model/"+prefix+"-"+cur_time+".h5"
model.save(model_path)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 128)          75264     
_________________________________________________________________
dropout (Dropout)            (None, 100, 128)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 128)          131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 128)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 128)          131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 128)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 100, 128)          1

In [10]:
# 7.1 visualize loss
keys = list(history_model.history.keys())
training_loss = history_model.history['loss']
test_loss = history_model.history['val_loss']
epoch_count = range(1, len(training_loss) + 1)
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Train', 'Test'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('model:'+prefix+'   batch_size:'+str(batch_size)+'   epochs:'+str(epochs)+'   units:'+str(units),loc ='left')
plt.tight_layout()
plt.grid()
plt.savefig('./data/img-nq/results/'+cur_time+'-loss')
plt.clf()

# 7.2 visualize accuracy
training_accuracy = history_model.history['accuracy']
test_accuracy = history_model.history['val_accuracy']
epoch_count = range(1, len(training_accuracy) + 1)
plt.plot(epoch_count, training_accuracy, 'r--')
plt.plot(epoch_count, test_accuracy, 'b-')
plt.legend(['Train', 'Test'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('model:'+prefix+'   batch_size:'+str(batch_size)+'   epochs:'+str(epochs)+'   units:'+str(units),loc ='left')
plt.tight_layout()
plt.grid()
plt.savefig('./data/img-nq/results/'+cur_time+'-accuracy')
plt.clf()

<Figure size 432x288 with 0 Axes>

In [11]:
# 8.0 model evaluate
verbose = 1
train_score = model.evaluate(x_train, y_train, verbose=verbose)
test_score = model.evaluate(x_test, y_test, verbose=verbose)
valid_score = model.evaluate(x_valid, y_valid, verbose=verbose)

print('Train Score: %.4f MSE (%.4f RMSE)' % (train_score[0], math.sqrt(train_score[0])))
print('Test Score: %.4f MSE (%.4f RMSE)' % (test_score[0], math.sqrt(test_score[0])))
print('Validate Score: %.4f MSE (%.4f RMSE)' % (valid_score[0], math.sqrt(valid_score[0])))

552/552 [==============================] - 5s 8ms/step - loss: 0.0025 - accuracy: 0.2882
Train Score: 0.0022 MSE (0.0471 RMSE)
Test Score: 0.0015 MSE (0.0393 RMSE)
Validate Score: 0.0025 MSE (0.0503 RMSE)


In [12]:
# 9.0 預測
predict1 = model.predict(x_valid)
df7 = pd.DataFrame(predict1, columns=['t1', 't2', 't3', 't4', 't5'])
date_valid2 = [v[0] for v in date_valid]
df7['udate'] = date_valid2
df7.index = df7['udate']
df7

,t1,t2,t3,t4,t5,udate
udate,,,,,,
2020-08-24 18:40:00,-0.238662,-0.244388,-0.241034,-0.241961,-0.249648,2020-08-24 18:40:00
2020-08-24 18:41:00,-0.232173,-0.237861,-0.234356,-0.235518,-0.243114,2020-08-24 18:41:00
2020-08-24 18:42:00,-0.226007,-0.231625,-0.228000,-0.229428,-0.236928,2020-08-24 18:42:00
2020-08-24 18:43:00,-0.220804,-0.226336,-0.222627,-0.224343,-0.231744,2020-08-24 18:43:00
2020-08-24 18:44:00,-0.216979,-0.222421,-0.218667,-0.220681,-0.227974,2020-08-24 18:44:00
...,...,...,...,...,...,...
2020-09-11 15:50:00,-0.588314,-0.595882,-0.594846,-0.600450,-0.603981,2020-09-11 15:50:00
2020-09-11 15:51:00,-0.590400,-0.598107,-0.596996,-0.602556,-0.606247,2020-09-11 15:51:00
2020-09-11 15:52:00,-0.592525,-0.600383,-0.599179,-0.604693,-0.608546,2020-09-11 15:52:00


In [17]:
# 10.0 
len_shape_y = x_valid.shape[2]-1
fill_list = list(repeat(0, len_shape_y))
df8 = pd.DataFrame()

# 10.1 逆向
for k, v in scalers.items():
    mask = ((df7['udate'] >= v['day_start']) & (df7['udate'] <= v['day_end']))
    df8_1 = df7.loc[mask]
    df8_2 = df8_1.drop(['udate'], axis=1)
    # 10.2
    data4 = []
    for k1, v1 in df8_2.iterrows():
        data4_1 = []
        for v2 in v1:
            data4_1.append([v2] + fill_list)
        data4_2 = v['scaler'].inverse_transform(data4_1)
        data4_3 = [v[0] for v in data4_2]
        data4.append(data4_3)
    # 10.3
    df8_3 = pd.DataFrame(data4, columns=['t1', 't2', 't3', 't4', 't5'])
    df8_3.index = df8_1['udate']
    # 10.4 合併
    if df8_3.shape[0] > 0:
        df8 = pd.concat([df8, df8_3], axis=0, join='outer', ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, copy=True)

df8['udate'] = df8.index.values
df8 = df8[['udate', 't1', 't2', 't3', 't4', 't5']]
# df8[df8.index.duplicated()]
df8

,udate,t1,t2,t3,t4,t5
udate,,,,,,
2020-08-24 18:40:00,2020-08-24 18:40:00,11645.262722,11641.448304,11643.682271,11643.065046,11637.943903
2020-08-24 18:41:00,2020-08-24 18:41:00,11649.585053,11645.796315,11648.131047,11647.357112,11642.296501
2020-08-24 18:42:00,2020-08-24 18:42:00,11653.692941,11649.950084,11652.364923,11651.414036,11646.417987
2020-08-24 18:43:00,2020-08-24 18:43:00,11657.158678,11653.473822,11655.944568,11654.801442,11649.871088
2020-08-24 18:44:00,2020-08-24 18:44:00,11659.706663,11656.081455,11658.582457,11657.240661,11652.382185
...,...,...,...,...,...,...
2020-09-11 15:50:00,2020-09-11 15:50:00,11067.461356,11065.050431,11065.380532,11063.595386,11062.470498
2020-09-11 15:51:00,2020-09-11 15:51:00,11066.796748,11064.341659,11064.695476,11062.924247,11061.748624
2020-09-11 15:52:00,2020-09-11 15:52:00,11066.119989,11063.616633,11064.000222,11062.243576,11061.016268


In [25]:
# 11.1
df9 = df3.drop(list(df3.columns)[6:], axis=1)
df9_1 = pd.concat([df9, df8], axis=1)

# 11.2 
start2, end2 =  date_valid2[0], date_valid2[-2:-1][0]
start3 = start2 - timedelta(minutes=t_pus_no+window_size)
mask3 = ((df9_1.index >= start3) & (df9_1.index <= start2)) # 窗口步長
mask4 = ((df9_1.index >= start2) & (df9_1.index <= end2)  & (df9_1['t1'] > 0)) # 預測
df9_2 = df9_1.loc[(mask3 | mask4)]

# 11.3 save
path_1 = os.path.abspath(os.path.join('data', 'nq', 'prediction', 'nq-prediction.csv'))
if os.path.exists(path_1):
    os.remove(path_1)
df9_3 = df9_2.drop(['udate'], axis=1)
df9_3.to_csv(path_1)

df9_3

,High,Low,Open,Close,Volume,t1,t2,t3,t4,t5
udate,,,,,,,,,,
2020-08-24 17:00:00,11636.00,11637.75,11633.00,11635.50,201,NaN,NaN,NaN,NaN,NaN
2020-08-24 17:01:00,11635.50,11637.00,11634.75,11635.75,68,NaN,NaN,NaN,NaN,NaN
2020-08-24 17:02:00,11635.75,11637.00,11635.75,11637.00,38,NaN,NaN,NaN,NaN,NaN
2020-08-24 17:03:00,11637.00,11640.50,11636.75,11639.75,129,NaN,NaN,NaN,NaN,NaN
2020-08-24 17:04:00,11639.75,11641.00,11638.25,11638.75,50,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2020-09-11 15:49:00,11064.75,11065.25,11063.25,11065.25,20,11068.061670,11065.689234,11066.001905,11064.205118,11063.128801
2020-09-11 15:50:00,11065.25,11066.25,11064.75,11066.25,18,11067.461356,11065.050431,11065.380532,11063.595386,11062.470498
2020-09-11 15:51:00,11066.25,11067.75,11065.00,11065.00,18,11066.796748,11064.341659,11064.695476,11062.924247,11061.748624
